In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Mahesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Mahesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement scann (from versions: none)
ERROR: No matching distribution found for scann

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Mahesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [5]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [6]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [7]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [8]:
unique_user_ids[:10]

array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107'], dtype=object)

In [9]:
user_ids_vocabulary=tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
# user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

In [10]:
movie_titles_vocabulary=tf.keras.layers.StringLookup(vocabulary=unique_movie_titles, mask_token=None)

In [11]:
import tensorflow_recommenders as tfrs

In [12]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [13]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [14]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
25/25 [==============================] - 6s 174ms/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0051 - factorized_top_k/top_50_categorical_accuracy: 0.0439 - factorized_top_k/top_100_categorical_accuracy: 0.0989 - loss: 33077.8412 - regularization_loss: 0.0000e+00 - total_loss: 33077.8412
Epoch 2/3
25/25 [==============================] - 4s 171ms/step - factorized_top_k/top_1_categorical_accuracy: 1.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0050 - factorized_top_k/top_10_categorical_accuracy: 0.0143 - factorized_top_k/top_50_categorical_accuracy: 0.1045 - factorized_top_k/top_100_categorical_accuracy: 0.2092 - loss: 31011.4446 - regularization_loss: 0.0000e+00 - total_loss: 31011.4446
Epoch 3/3
25/25 [==============================] - 4s 174ms/step - factorized_top_k/top_1_categorical_accuracy: 3.5000e-04 - factorized_top_k/top_5_categorica

In [15]:
new_user_movie_titles = ["The Matrix", "Inception", "The Dark Knight"]
new_user_user_ids = ["new_user_123", "new_user_123", "new_user_123"]
new_user_ratings = [5, 4, 3]

In [17]:
ratings_with_ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

In [18]:
# Convert the separate lists to tensors
new_user_ratings_dataset = tf.data.Dataset.from_tensor_slices({
    "movie_title": tf.constant(new_user_movie_titles),
    "user_id": tf.constant(new_user_user_ids)
})

In [19]:
updated_ratings = ratings_with_ratings.concatenate(new_user_ratings_dataset)

In [20]:
model.fit(updated_ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 4s 178ms/step - factorized_top_k/top_1_categorical_accuracy: 4.3999e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0104 - factorized_top_k/top_10_categorical_accuracy: 0.0279 - factorized_top_k/top_50_categorical_accuracy: 0.1758 - factorized_top_k/top_100_categorical_accuracy: 0.3129 - loss: 30076.2562 - regularization_loss: 0.0000e+00 - total_loss: 30076.2562
Epoch 2/3
25/25 [==============================] - 4s 176ms/step - factorized_top_k/top_1_categorical_accuracy: 3.1999e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0110 - factorized_top_k/top_10_categorical_accuracy: 0.0304 - factorized_top_k/top_50_categorical_accuracy: 0.1976 - factorized_top_k/top_100_categorical_accuracy: 0.3449 - loss: 29819.6561 - regularization_loss: 0.0000e+00 - total_loss: 29819.6561
Epoch 3/3
25/25 [==============================] - 5s 180ms/step - factorized_top_k/top_1_categorical_accuracy: 4.0999e-04 - factorized_top_k/top_5_categorica

In [24]:
_, recommended_titles = index(np.array(["new_user_123"]))
print(f"Top 3 recommendations for new_user_123: {recommended_titles[0, :20]}")

Top 3 recommendations for new_user_123: [b'Catwalk (1995)' b'Boys in Venice (1996)' b'Stranger, The (1994)'
 b'Mighty, The (1998)'
 b'Day the Sun Turned Cold, The (Tianguo niezi) (1994)'
 b'Nil By Mouth (1997)' b'Nosferatu a Venezia (1986)'
 b'Relative Fear (1994)' b'Amityville: Dollhouse (1996)'
 b'Jack and Sarah (1995)']
